In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import glob
import toml
from segments import SegmentsClient
from tqdm.auto import tqdm
from markovids import vid

1. get labels from fused dataset
2. create a dataset for each annotator...
3. compare against fluorescence and across humans...

## User functions

In [5]:
# https://stackoverflow.com/questions/52498777/apply-matplotlib-or-custom-colormap-to-opencv-image
def mpl_to_cv2_colormap(cmap_name):
    import matplotlib.pyplot as plt

    cmap = plt.get_cmap(cmap_name)
    sm = plt.cm.ScalarMappable(cmap=cmap)

    # step from min to max, strip alpha, rgb to bgr
    color_range = sm.to_rgba(np.linspace(0, 1, 256), bytes=True)[:, 2::-1]
    return color_range.reshape(256, 1, 3)


def minmax_scale(batch_frames):
    use_min = batch_frames.min(axis=(1, 2), keepdims=True)
    use_max = batch_frames.max(axis=(1, 2), keepdims=True)

    # normalize between 0-1
    batch_frames = (batch_frames - use_min) / (use_max - use_min)
    batch_frames[batch_frames < 0] = 0
    batch_frames[batch_frames > 1] = 1
    return batch_frames


def intensity_to_rgba(frame, minval=1800, maxval=2200, colormap=cv2.COLORMAP_TURBO):
    disp_frame = frame.copy().astype("float")
    disp_frame -= minval
    disp_frame[disp_frame < 0] = 0
    disp_frame /= np.abs(maxval - minval)
    disp_frame[disp_frame >= 1] = 1
    disp_frame *= 255
    bgr_frame = cv2.applyColorMap(disp_frame.astype(np.uint8), colormap)
    rgb_frame = cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2RGB)

    # new_frame[:, :, :3] = rgb_frame / 255.0
    return rgb_frame

# Load in metadata

In [6]:
base_dir = "/storage/home/hcoda1/4/jmarkowitz30/shared_folder/active_lab_members/markowitz_jeffrey/active_projects/quantum_dots/timecourse_02/"
# base_dir = "/mnt/data/jmarkow/projects_active/mouse_open_field_multicam_lucid"
# base_dir = "/home/jmarkow/data_dir/active_projects/quantum_dots/timecourse_02_joints/"
background_path = "_bground"
frame_path = "_proc"

In [7]:
calibration_data_file = os.path.join(os.path.dirname(base_dir), "../timecourse_02_calibration_v2.toml")
calibration_data = toml.load(calibration_data_file)
# calibration_data = toml.load("/home/jmarkow/data_dir/active_projects/quantum_dots/timecourse_02_calibration_v2.toml")
cameras = sorted(list(calibration_data["intrinsics"].keys()))

In [8]:
# camera_dirs = [os.path.join(base_dir, _camera) for _camera in cameras]

In [95]:
minval = 0
maxval = 255
use_cmap = plt.matplotlib.cm.viridis
use_cmap = mpl_to_cv2_colormap(use_cmap)

pad = 0  # pad with a 20px border
dry_run = False

# Get what already exists on segments

In [96]:

# You can find your api key at https://segments.ai/account
api_key = os.getenv("SEGMENTSAPI")
client = SegmentsClient(api_key)

In [97]:
dataset_name = "jmarkow/basler-nir-plexiglass-arena-keypoints-fused-round2"
# dataset_name = "jmarkow/basler-nir-plexiglass-arena-keypoints-fused-kneejoints"
samples = client.get_samples(dataset_name, label_status=["LABELED", "REVIEWED"])

In [129]:
root_dir = "/storage/home/hcoda1/4/jmarkowitz30/shared_folder/active_lab_members/markowitz_jeffrey/active_projects/quantum_dots/"
export_dir = "/storage/home/hcoda1/4/jmarkowitz30/shared_folder/active_lab_members/markowitz_jeffrey/active_projects/keypoints_basler_nir_plexiglass_arena/segments_ai_labeling_export_fused_round2_reflect_only/"

In [130]:
used_files = [_sample.metadata["dat_path_reflect"] for _sample in samples]

In [131]:
uniq_files = sorted(list(set(used_files)))

# Export data to directory first

In [132]:
n_dirlevels = 4

In [133]:
from PIL import Image
from PIL.PngImagePlugin import PngInfo
import warnings

In [134]:
os.makedirs(export_dir, exist_ok=True)

In [135]:
for _file in tqdm(uniq_files):
    use_samples = [_sample for _sample in samples if _sample.metadata["dat_path_reflect"] == _file]
    use_frames = [_sample.metadata["frame_index"] for _sample in use_samples]
    use_fnames = [os.path.basename(_sample.metadata["img_path"]) for _sample in use_samples]

    path_suffix = os.path.join(*_file.split(os.sep)[-n_dirlevels:])
    use_file = os.path.join(root_dir, path_suffix)

    reflect_reader = vid.io.AutoReader(use_file)
    reflect_frames = reflect_reader.get_frames(use_frames)
    reflect_reader.close()

    for (
        _sample,
        _fname,
        _im,
    ) in zip(use_samples, use_fnames, reflect_frames):
        use_frame = np.pad(_im, ((pad, pad), (pad, pad)), mode="constant", constant_values=0)
        export_frame = intensity_to_rgba(use_frame, minval=minval, maxval=maxval, colormap=use_cmap)[:, :, :3]

        filename_export = os.path.join(export_dir, os.path.basename(_fname))

        targetImage = Image.fromarray(export_frame)
        metadata = PngInfo()
        metadata.add_text("pad", str(pad))
        metadata.add_text("frame_index", str(_sample.metadata["frame_index"]))
        metadata.add_text("camera", _sample.metadata["camera"])
        metadata.add_text("dat_path_fluo", _sample.metadata["dat_path_fluo"])
        metadata.add_text("dat_path_reflect", _sample.metadata["dat_path_reflect"])
        metadata.add_text("img_path", filename_export)
        metadata.add_text("original_segments_uuid", _sample.uuid)
        metadata.add_text("original_segments_name", _sample.name)
        metadata.add_text("original_img_path", _sample.metadata["img_path"])

        if not dry_run:
            if not os.path.exists(filename_export):
                targetImage.save(filename_export, pnginfo=metadata)
            else:
                warnings.warn(f"{filename_export} already exists, skipping...")
        # export if we're not doing a dry run
        #

  0%|          | 0/60 [00:00<?, ?it/s]

In [139]:
filenames_exported = sorted(glob.glob(os.path.join(export_dir, "*.png")))

# Upload new data to segments.ai

In [ ]:
urls = []

In [146]:
# assume everything is synced...

In [151]:
root_dataset_name = "jmarkow/basler-nir-plexiglass-arena-keypoints-fused-round2-reflectonly"
labelers = ["zeynep", "amartya", "claire", "brittany"]

In [166]:
current_samples = client.get_samples(f"{root_dataset_name}-{labelers[0]}")

In [167]:
filenames_uploaded = []
for _sample in current_samples:
    filenames_uploaded.append(os.path.basename(_sample.metadata["img_path"]))

In [168]:
filenames_to_upload = [_fname for _fname in filenames_exported if os.path.basename(_fname) not in filenames_uploaded]

In [163]:
urls = []
for _filename in tqdm(filenames_to_upload):
    with open(_filename, "rb") as f:
        use_filename = os.path.basename(_filename)
        asset = client.upload_asset(f, use_filename)
    image_url = asset.url
    urls.append(image_url)

  0%|          | 0/592 [00:00<?, ?it/s]

In [164]:
for _name, _url in tqdm(zip(filenames_to_upload, urls), total=len(filenames_to_upload)):
    # convert numeric metadata prior to adding to dataset...
    metadata = Image.open(_name).text
    metadata["pad"] = int(metadata["pad"])
    metadata["frame_index"] = int(metadata["frame_index"])
    attributes = {"image": {"url": _url}}
    for _labeler in labelers:
        metadata["labeler"] = _labeler
        use_dataset_name = f"{root_dataset_name}-{_labeler}"
        sample_name = f"{os.path.basename(_name)}-{_labeler}" 
        sample = client.add_sample(use_dataset_name, sample_name, attributes, metadata=metadata)

  0%|          | 0/592 [00:00<?, ?it/s]

In [ ]:
# samples = client.get_samples(dataset_name, per_page=10000, label_status=["UNLABELED"])